In [1]:
import pandas as pd
import numpy as np 
from functions.Strategy import *
from functions.UtilsCreateDataFrame import *
import time

C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\giova\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
path = "H:\AllTickers_klines_2020-12_2021-12.csv.gz"
coin_df = loadCleanDataFrame(path)
#let's test two months
coin_df

,ETH,BNB,XRP,BTC,ADA,SOL,DOGE,TRX,BUSD,MATIC,LTC,DOT,USDC
time,,,,,,,,,,,,,
2020-12-01 00:00:59.999000+00:00,616.280,31.49085,0.665205,19705.900,0.172355,1.9665,0.003559,0.032380,0.99960,0.019620,87.735,5.37605,0.99960
2020-12-01 00:01:59.999000+00:00,615.520,31.48720,0.665410,19692.585,0.172525,1.9653,0.003560,0.032385,0.99960,0.019590,87.720,5.36925,0.99955
2020-12-01 00:02:59.999000+00:00,616.095,31.58415,0.665940,19682.370,0.173210,1.9655,0.003566,0.032415,0.99965,0.019595,87.735,5.37255,0.99955
2020-12-01 00:03:59.999000+00:00,616.945,31.67950,0.667905,19673.010,0.174180,1.9706,0.003572,0.032495,0.99970,0.019630,87.730,5.37720,0.99955
2020-12-01 00:04:59.999000+00:00,615.795,31.65380,0.666910,19626.910,0.173875,1.9633,0.003569,0.032500,0.99960,0.019590,87.540,5.37010,0.99950
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:55:59.999000+00:00,3678.005,511.50000,0.831300,46269.240,1.309000,170.2850,0.170300,0.075420,0.99995,2.521500,146.250,26.69000,0.99990
2021-12-31 23:56:59.999000+00:00,3676.865,511.60000,0.830750,46251.335,1.309000,170.2050,0.170250,0.075410,0.99990,2.523000,146.300,26.69000,0.99995
2021-12-31 23:57:59.999000+00:00,3677.270,511.65000,0.830200,46249.175,1.309500,170.1800,0.170350,0.075440,0.99990,2.525500,146.400,26.69000,1.00000


In [5]:
%%time
# calibration 12 hours 
#asset 1: ADA
#asset 2: ETH

cal_w1 = {"hours":12}
beta_table = create_beta_table(
    coin_df = coin_df,
    asset_name_1 = "ADA",
    asset_name_2 = "ETH",
    calibration_window = cal_w1,
    frequency={"minutes": 1},
    safe_output_csv=True,
    n_job=4,
    output_folder = "H:",
    stat_test = "adfuller"
)


Wall time: 1h 39s


In [3]:
%%time 

tr_w6 = {"hours":12}
tr_w5 = {"hours":6}
tr_w4 = {"hours" : 4}
tr_w3 = {"hours":2}
tr_w2 = {"hours":1}
tr_w1 = {"minutes":30}
tresh = [tr_w1, 
        tr_w2, 
        tr_w3, 
        tr_w4, tr_w5, 
        tr_w6]


cal_w1 = {"hours":12}


result = getCombRet(
    coin_df = coin_df,
    asset_name_1 = "ADA",
    asset_name_2 = "ETH",
    trading_windows = tresh,
    calib_window = cal_w1,
    p_values = [0.15, 0.1, 0.05, 0.01],
    stop_loss=0.2,
    stat_test = "adfuller",
    safe_beta_csv=True,
    input_folder = "H:",
    output_folder_beta = "H:"
)

C:\Users\giova\StatisticalArbitrage\functions\Strategy.py:124: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  coin_df["Month"] = coin_df.index.to_period("M")


Beta Table Loaded


C:\Users\giova\StatisticalArbitrage\functions\Strategy.py:139: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  beta_df["Month"] = beta_df.index.to_period("M")


Wall time: 30min 37s


In [5]:
import joblib
joblib.dump(result, "ADA_ETH.joblib")

['ADA_ETH.joblib']

In [20]:
result["2021-12"][1]

,0.15,0.10,0.05,0.01
0 days 00:30:00,0.000025,0.001747,0.000589,0.04363
0 days 01:00:00,0.0,0.000006,0.065437,0.033348
0 days 02:00:00,0.0,0.468888,0.000002,0.02772
0 days 04:00:00,0.010607,0.001262,0.000006,0.522295
0 days 06:00:00,0.0,0.0,0.0,0.431575
0 days 12:00:00,0.0,0.0,0.000004,0.165529


In [21]:
#we also keep track of the trading books 
result["2021-01"][1][f"{tr_w3}_{0.1}"]

,ret_,ret_no_fee,ret_type,spread_enter,spread_exit,fee_enter,fee_exit,borrow_fee,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est,Month
0,0.976769,0.907756,Short,-0.011034,-0.012051,0.000379,0.000378,0.000004,2021-01-01 02:15:59.999000+00:00,2021-01-01 04:03:59.999000+00:00,0.000262,-0.013032,0.634686,0.000880,7.158626e-17,0.032286,2021-01-01 02:02:59.999000+00:00,2021-01
1,1.011574,0.964754,Long,-0.055007,-0.053069,0.000412,0.000412,0.001751,2021-01-01 07:26:59.999000+00:00,2021-01-01 08:07:59.999000+00:00,0.000316,-0.052765,0.630478,0.000891,-9.278843e-17,0.012022,2021-01-01 06:06:59.999000+00:00,2021-01
2,1.013899,0.984298,Long,-0.089671,-0.088263,0.000450,0.000448,0.001757,2021-01-01 11:32:59.999000+00:00,2021-01-01 12:10:59.999000+00:00,0.000365,-0.088391,0.831973,0.000814,-3.276700e-18,0.004761,2021-01-01 10:55:59.999000+00:00,2021-01
3,0.991112,0.982116,Short,-0.103730,-0.105585,0.000464,0.000467,0.000002,2021-01-01 13:00:59.999000+00:00,2021-01-01 13:47:59.999000+00:00,0.000387,-0.105296,0.867305,0.000710,-9.806970e-17,0.015081,2021-01-01 12:57:59.999000+00:00,2021-01
4,1.010631,1.001525,Short,-0.103226,-0.103069,0.000469,0.000469,0.000002,2021-01-01 14:40:59.999000+00:00,2021-01-01 14:58:59.999000+00:00,0.000387,-0.105296,0.867305,0.000710,-9.806970e-17,0.015081,2021-01-01 12:57:59.999000+00:00,2021-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0.950020,0.960233,Short,0.113431,0.117941,0.000572,0.000579,0.000007,2021-01-30 12:00:59.999000+00:00,2021-01-30 13:36:59.999000+00:00,0.000171,0.111126,0.858724,0.001166,-3.137922e-17,0.001216,2021-01-30 11:35:59.999000+00:00,2021-01
234,1.180393,1.165946,Short,-0.115281,-0.096150,0.000823,0.000839,0.000004,2021-01-30 16:20:59.999000+00:00,2021-01-30 16:57:59.999000+00:00,0.000344,-0.120481,0.568820,0.002329,8.249574e-18,0.096403,2021-01-30 14:56:59.999000+00:00,2021-01
235,1.010952,1.002384,Short,-0.225630,-0.225092,0.000967,0.000963,0.000004,2021-01-31 00:28:59.999000+00:00,2021-01-31 00:54:59.999000+00:00,0.000435,-0.237715,0.631610,0.004828,3.068533e-17,0.099154,2021-01-30 22:53:59.999000+00:00,2021-01
236,0.979015,0.963564,Short,-0.109910,-0.113915,0.000850,0.000841,0.000008,2021-01-31 00:57:59.999000+00:00,2021-01-31 02:56:59.999000+00:00,0.000351,-0.121622,0.333230,0.005367,-3.808682e-17,0.057476,2021-01-31 00:55:59.999000+00:00,2021-01


In [6]:
result["2020-12"][0]

,0.10,0.05
0 days 00:30:00,1.072182,1.432628
0 days 02:00:00,0.828593,0.000483
0 days 01:00:00,0.095504,0.162217


In [11]:
#we also keep track of the trading books 
result["2020-12"][1][f"{tr_w3}_{0.1}"]

,ret_,ret_no_fee,ret_type,spread_enter,spread_exit,fee_enter,fee_exit,borrow_fee,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est,Month
0,0.964669,0.973561,Short,0.765435,0.785672,0.003340,0.003372,0.000094,2020-12-02 01:29:59.999000+00:00,2020-12-02 01:48:59.999000+00:00,42.316035,0.704664,0.327890,0.021620,2.109424e-16,0.098972,2020-12-02 01:17:59.999000+00:00,2020-12
1,1.013640,1.020304,Short,0.962048,0.942514,0.003174,0.003143,0.000094,2020-12-02 02:23:59.999000+00:00,2020-12-02 02:50:59.999000+00:00,36.235337,0.900653,0.209353,0.024978,4.058482e-16,0.092561,2020-12-02 02:19:59.999000+00:00,2020-12
2,0.987296,1.000690,Short,0.556446,0.556062,0.003682,0.003673,0.000097,2020-12-02 16:44:59.999000+00:00,2020-12-02 17:08:59.999000+00:00,50.779857,0.499718,0.197281,0.031227,2.763222e-16,0.089289,2020-12-02 16:37:59.999000+00:00,2020-12
3,0.998708,1.005228,Short,1.015088,1.009781,0.003259,0.003263,0.000098,2020-12-02 17:14:59.999000+00:00,2020-12-02 17:40:59.999000+00:00,36.457974,0.944598,0.078856,0.032483,2.220446e-16,0.011555,2020-12-02 17:09:59.999000+00:00,2020-12
4,0.995183,0.997509,Short,2.008185,2.013188,0.002283,0.002290,0.000098,2020-12-02 17:48:59.999000+00:00,2020-12-02 18:12:59.999000+00:00,4.430770,1.938501,0.000714,0.033964,-1.085551e-16,0.002319,2020-12-02 17:41:59.999000+00:00,2020-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,0.982310,0.996654,Long,0.383857,0.382573,0.002746,0.002759,0.000001,2020-12-30 08:29:59.999000+00:00,2020-12-30 08:40:59.999000+00:00,44.147407,0.436439,0.178984,0.025642,1.430954e-16,0.091727,2020-12-30 08:09:59.999000+00:00,2020-12
101,1.002482,1.006179,Long,1.075593,1.082239,0.001984,0.001991,0.000001,2020-12-30 22:57:59.999000+00:00,2020-12-30 23:17:59.999000+00:00,16.952465,1.113034,0.022474,0.011729,-2.929755e-17,0.088995,2020-12-30 22:46:59.999000+00:00,2020-12
102,1.006203,1.000892,Long,-1.787328,-1.788922,0.004745,0.004747,0.000001,2020-12-31 01:50:59.999000+00:00,2020-12-31 01:53:59.999000+00:00,122.925409,-1.752612,0.419468,0.017388,-5.350658e-16,0.093520,2020-12-31 01:22:59.999000+00:00,2020-12
103,1.010708,1.006731,Long,-3.024017,-3.044372,0.006017,0.006008,0.000001,2020-12-31 03:05:59.999000+00:00,2020-12-31 03:29:59.999000+00:00,168.705621,-2.987579,0.683290,0.018375,3.552714e-16,0.088807,2020-12-31 02:58:59.999000+00:00,2020-12
